In [1]:
import torch.nn as nn
import torchvision
import torch.utils.data
from models import BasicParkingNet

In [2]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(),
        torchvision.transforms.Resize((40, 60)),
        torchvision.transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=32,
    shuffle=True
)

In [3]:
# Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BasicParkingNet().to(device).train()
criterion = nn.BCEWithLogitsLoss()
num_episodes = 100

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        model.optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        model.optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../models/basic_parking_net.pth")
        print("\tSaving!")


Episode 0, loss: 54.49822489917278
	Saving!
Episode 1, loss: 23.61001396039501
	Saving!
Episode 2, loss: 17.79724326566793
	Saving!
Episode 3, loss: 12.002940913429484
	Saving!
Episode 4, loss: 18.019632218405604
Episode 5, loss: 13.80506778229028
Episode 6, loss: 13.95336301671341
Episode 7, loss: 12.822345243301243
Episode 8, loss: 11.437574292183854
	Saving!
Episode 9, loss: 27.256021168548614
Episode 10, loss: 25.580116724479012
Episode 11, loss: 16.225075623602606
Episode 12, loss: 13.793388866819441
Episode 13, loss: 11.325651940191165
	Saving!
Episode 14, loss: 26.166311000473797
Episode 15, loss: 13.762813700828701
Episode 16, loss: 8.113424542360008
	Saving!
Episode 17, loss: 36.836478646378964
Episode 18, loss: 11.343096978031099
Episode 19, loss: 76.1568021774292
Episode 20, loss: 44.946558732539415
Episode 21, loss: 11.054543118458241
Episode 22, loss: 11.613666731165722
Episode 23, loss: 8.593197119887918
Episode 24, loss: 16.174920707242563
Episode 25, loss: 9.41160342469